In [1]:
import sys
sys.path.append('/home/insign/Doc/insign/Python_utils/torch2trt')
from torch2trt import torch2trt
from torch2trt import TRTModule
import torch
import datetime, time, copy, yaml
device = torch.device('cuda')
from copy import deepcopy
import sys, os, time
sys.path.append('/home/insign/Doc/insign/Mask_yolo')
import torch
import cv2
import numpy as np 
from model.od.data.datasets import letterbox
from typing import Any
from model.backbone_YOLO import *
from model.head_RCNN import *
from model.groundtrue_import import *
from PIL import Image
from torchvision import transforms

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
def image_loading(img_path):
    image = cv2.imread(str(img_path))
    img_h, img_w = image.shape[0], image.shape[1]
    image = letterbox(image, new_shape=640)[0]
    im0s = deepcopy(image)
    image = image[:, :, ::-1].transpose(2, 0, 1)
    image = np.ascontiguousarray(image)
    image = torch.from_numpy(image).unsqueeze(0).to(device)
    image = image.float()
    image /= 255.0
    return image, im0s, img_h, img_w

image, im0s, img_h, img_w = image_loading('/home/insign/Doc/insign/Mask_yolo/Polyp/Images/2022012001_33.jpg')

yolo_cfg = '/home/insign/Doc/insign/Mask_yolo/config/config.yaml'
with open(yolo_cfg, 'r') as f:
    cfg = yaml.load(f, Loader=yaml.Loader)
device = torch.device('cuda')
yolo = model_manipulate(cfg['model']['weight']).eval().to(device)
x = image
pred = yolo(x)
rois = non_max_suppression(pred['rois'][0],cfg['nms']['conf_thres'], cfg['nms']['iou_thres'], classes= cfg['nms']['classes'],agnostic=cfg['nms']['agnostic_nms'])
boxes = rois[0][:,:4]
feature_map = pred['feature_map']



/usr/local/lib/python3.8/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
from turtle import forward
from torchvision.ops import MultiScaleRoIAlign
from torch import nn

class ROIAlign(nn.Module):
    def __init__(self):
        super().__init__()
        self.featurealign1 = nn.Conv2d(256, 512, 1, 1).to('cuda')
        self.featurealign2 = nn.Conv2d(512, 512, 1, 1).to('cuda')
        self.featurealign3 = nn.Conv2d(1024, 512, 1, 1).to('cuda')
        self.mask_roi_pool = MultiScaleRoIAlign(featmap_names=["feat1","feat2","feat3"] ,output_size=28, sampling_ratio=2)
        
    def featuremapPack(self, feat1, feat2, feat3):
        from collections import OrderedDict
        map = {}
        key_name = ["feat1","feat2","feat3"]
        feature_map = [feat1,feat2,feat3]
        for i,j in zip(key_name, feature_map):
            map[i] = j
        return map
    
    def forward(self, feat1, feat2, feat3, box):
        features_0 = self.featurealign1(feat1)
        features_1 = self.featurealign2(feat2)
        features_2 = self.featurealign3(feat3)
        featuremap = self.featuremapPack(features_0, features_1, features_2)
        # featurepooling = self.mask_roi_pool(featuremap, [box], image_shapes=[(640,640)])
        return features_2

model = ROIAlign()
# featuremap = model(feature_map[0], feature_map[1], feature_map[2], boxes)
model_trt = torch2trt(model, [feature_map[0], feature_map[1], feature_map[2], boxes])

In [8]:
class test(nn.Module):
    def __init__(self, min, max):
        super().__init__()
        self.min = min
        self.max = max
    def forward(self, x):
        y=torch.clamp(x, self.min, self.max)
        return y

x = torch.rand(1,1)
clamp = test(min=-0.5, max=0.5)
y = clamp(x)
model_trt = torch2trt(clamp,[x])

tensor([[0.10804]]) *******************
tensor([[0.10804]]) **********


[TensorRT] ERROR: Tensor: output_0 trying to set to TensorLocation::kHOST but only kDEVICE is supported (only network inputs may be on host)
[TensorRT] ERROR: Tensor: input_0 set to TensorLocation::kHOST but only kDEVICE is supported (only RNNv2 allows host input)
[TensorRT] ERROR: Network validation failed.


AttributeError: 'float' object has no attribute 'shape'